Databases contains collections which contains documents

Format to access collection ---> <Database_name>.<Collection_name>

### Commands

help - to show commands

db

show dbs - show all available databases

use <database_name> - switch database and hold the reference to db variable

mongod -dbpath=F:\Engineering\installedTools\MongoDB\Server\4.0\data

db.movies.insertOne({})-
    insert a document in movies collection in the current db (if db is not created yet, it will create it first and then add document to a given collection)
    it will return a document only (json) having information of acknowledgement and inserted id (_id is must for every document and if not provided mongo will create it by it's own)
        By default index is created on _id field and we have to manually create other indexes on other columns.
![12_MOngoDB_id.JPG](img/12_MOngoDB_id.JPG)
    

We can copy a record from compass, which doesn't show _id field in the copy prompt.

#### Bulk Inserts - Ordered inserts [default - stop as and when it encounters error] or Unordered inserts 
db.movies.insertMany([{<jsonData1>}, {<jsonData2>}])
            or
db.movies.insertMany([{<jsonData1>}, {<jsonData2>}], {ordered:false})


db.movies.find() 
    or 
db.movies.find().pretty() 
    or 
db.movies.find({}).pretty() 
    or 
db.movies.find({"title":"Kick"}).pretty() - 
    returns a cursor object not array. (hasNext & next methods)
       - can even filter keys with given value in nested documents.


show collections

db.moviesScratch.insertOne({title:"Star Trek II", year:1982, imdb:"tt0084726"})


db.moviesScratch.drop() # to drop this collection

    
Using Campus:
    1. {"birth year": {"$gte":1985, "$lt":2001} }
    
    
db.names.save - opens the javascript function defination

var j = db.name.find() or db.name.findOne()
j.name='Aman'
db.name.save(j) - will update the value of cell in the tuple

Handling Arrays:
1) On the entire Array.
2) Based on any element
3) based on a specific element
4) More Complex matches using operators
db.movies.find({cast:'Jeff Bridges'}).pretty() # All documents will pass who has "Jeff Bridges" cast array (any position)
db.movies.find({"cast.0":'Jeff Bridges'}).pretty() # All documents will pass who has "Jeff Bridges" in cast array at index 0 [Main actor of movie]
db.movies.find({cast:['Jeff Bridges', 'Tim Robbins']}).pretty() # will search exact match (size, order of elements both are important)

db.movies.find().pretty().count() # would return count even if we remove pretty from between.



### Cursors: (can also override the batch size)
MongoDB Server returns results to it's clients in batches (batch size would be 101 documents or documents having less than or equal to 1 megabyte data - subsequent batches would be 4 MB). By default shell iterates the cursor 20 times if response not stored in variable.
var c = db.movieDetails.find()
var doc= function() {return c.hasNext() ? c.next(): null;}
c.objsLeftInBatch(); # to check how many objects are in the batch
now repeatedly call doc() functions to iterate.
now call c.objsLeftInBatch() again and the count would be reduced now.

### Projections:
Avoid network over-head and processing requirement etc. by limiting the fields.
db.movieDetails.find({"genres.1":"Western"},{title:1}) # title + id (by default)
db.movieDetails.find({"genres.1":"Western"},{title:1, _id:0}) # only title 
0 means exclude
1 means include
So if we just need to remove 2 fields out of 1000s then simply mention these 2 fields with 0 value in second parameter.

### Update (updateOne will update the 1st it encounters) - can be used for nested fields as well
db.movieDetails.updateOne({title:"El Topo"}, {$set: {poster1:"AmandeepAddedThis"}}) # 1st is filter, 2nd is setter
Update Operators: $addToSet, $pullAll, $each etc for arrays and $inc, $set, $unset etc for other scenarios.
db.movieDetails.updateOne( {title: "The Martian"}, 
{$push: {reviews: { #push would make sure that 
    $each: [  # it's like flatArray in Java 8 stream, if not applied it takes complete array as one element and push it in the document
                {
                    rating: 0.5,
                    date: ISODate(2016-06-12T07:00:00Z),
                    reviewer: "Shannon B.",
                    text: "Enjoyed watching with my kids!"
                }
            ],
     $slice: 5 # to keep only 5 records(discards old ones),
     $position: 0 # to push in the very starting of the array
    
}}} )


db.movieDetails.updateMany({rating:null}, {$unset:{rating:""}}) # "" value doesn't matter here because operator is unset.

UpSerts: (update query which inserts the value if no value is found)
detail = db.movieDetails.findOne({"imdb.id":"tt4368814"})
db.movieDetails.updateOne(
    {
        "imdb.id":detail.imdb.id
    }, 
    {
        $set: detail
    },
    {
        upsert:true
    }
) # this query ensures that it wouldn't create a separate document, just one document having this imdb.id.


#### To replace an entire document
db.movieDetails.replaceOne({<filter>}, <newDocument>)
#why: you can save an object in python or javascript program and make hell lot of changes and then give it to server/mongodb to just replace it. If upsert flag is true, it just save new element (in case of update_one or update_many, it adds the filter as well)


# as for updates
db.movieDetails.deleteOne()
db.movieDetails.deleteMany()


### Query selectors
#### Comparision: $eq, $gt, $gte, $lt, $lte, $in, $nin
db.movieDetails.find({runtime: {$gt:90} } )
db.movieDetails.find({runtime: {$gt:90, $lte:120} }, {title:1} )
db.movieDetails.find({runtime: {$gt:90, $lte:120}, rated:{$ne: "UNRATED"} }, {title:1} )
db.movieDetails.find({rated:{$in: ["UNRATED", "PG"]} }, {title:1} )

#### Element: $exists, $type
db.movieDetails.find({"tomoto.meter": {$exists:true} } )
db.moviesDetails.find({"_id": {$type:"string"}}) # used in case same field have different types, but ideally it should not be the case.

Now consider a scenario where for some values of a key, we have value stored as null (without quotes) and we have few other records who doesn't have any such key. Now we want to make a query which fetches both of these records. To cater this we can use following query:
db.movieDetails.find({"tomoto.meter":null})

#### Logical: $or (takes array), $and, $not, $nor
db.movieDetails.find({ $or: [{"tomoto.meter": {$exists:true} }, {"metacritic": {$gt: 88}}]} )
Ok That's great about OR operator but why we need AND operator, after all it is equivalent to what we give in simple find method:
Actually in simple find method call, we can't specify the same key more than once (as the 'where' clause is actually a json, so last element will override the duplicate key) but that can be achieve as shown below:-
db.movieDetails.find({ $and: [ {"metacritic": {$ne: null}}, {"metacritic":{$exists:true}} ] })

#### Regex Operator: $regex (/slashes/ signifies the beginning and end of the regex expression)
db.movieDetails.find( { "awards.text": {$regex: /^Won.*/} } )
db.movieDetails.find( { "awards.text": {$regex: /^Won\s.*/} } )
^ means very beginning of text we want to match the next character (next to ^) = case-sensitive
. = match any character
* = any number of times of previous character




#### Array Operator: $all, $size, $elemMatch
db.movieDetails.find( { genres: {$all: ["Comedy", "Crime"]} } ) # all exist in queried array
db.movieDetails.find( {countries: {$size: 1} } )
For scenarios where there is an array of json objects and we want to satisfy 2 conditions for every json element like below:-
Data = boxOffice: [ 
                    {"country": "USA", "revenue":41.3} ,
                    {"country": "Australia", "revenue":2.9} ,
                    {"country": "UK", "revenue":10.1} ,
                    {"country": "Germany", "revenue":4.3} ,
                  ]
FirstTry =  db.movieDetails.find({boxOffice: {country:"UK", revenue:{$gt:15}}})
SecondTry =  db.movieDetails.find({boxOffice.country:"UK", boxOffice.revenue:{$gt:15}})

CorrectTry = db.movieDetails.find({boxOffice: {$elematch { country:"UK", revenue: {$gt:15} }}})


Run JS via mongo Shell:
mongo < myJavascript.js


type reddit.json | python -m json.tool



> db.movieDetails.find({$or: [    { 'awards.wins':{$gt:0} }  ,  {'awards.oscars':{$size: {$gt:0} } }  ]})
Error: error: {
        "ok" : 0,
        "errmsg" : "$size needs a number",
        "code" : 2,
        "codeName" : "BadValue"
}
db.movieDetails.find({$or: [    { 'awards.wins':{$gt:0} }  ,  {'awards.oscars.0':{$exists: true } }  ]})



db.movieDetails.updateMany({ year: {$gte: 2010, $lte: 2013}, "imdb.votes": {$lt: 10000},"tomato.consensus": null },
                           { $unset: { "tomato.consensus": "" } });
                           
db.movieDetails.updateMany({ year: {$gte: 2010, $lte: 2013}, "imdb.votes": {$lt: 10000},"tomato.consensus": null }, 
                           { $unset: { "tomato.censensus": "" } });

# Mongo DB schema Design

Although mongo-db is schema-less, but if we don't look much at schema it will impact the performance.

Now talking about the relational database, we know that during application development for schema design, we try to keep all tables atleast at 3rd-Normal form. In Mongo-DB we can keep data in 3rd-Normal Form.

But in mongodb, we keep schema considering the principle of "Application Driven Schema". We think about:
* What pieces of data are used together.
* What pieces of data are mostly read-only.

Accordingly we design the mongodb schema to suit the application data access patterns. We try to keep data in a way that is agnostic to the application.

Basic Facts of MongoDB:
* supports rich documents (meaning not only tabular data, but arrays of values, arrays of documents, nested documents etc)
* allows pre-join of data/ embed data
* Doesn't support join directly inside the kernel (if we need this we need to join in the application itself)
    * the reason is that joins are hard to scale.
* No constraints (not as important due to embedding)
* Don't support transactions but do support atomicity (we would discuss how to design our schema in order to support atomicity)
* No declared schema, but there is a good chance that your application is going to have a schema. 



Goals of normalization in relational databases:
* Free the database from modification anamolies. # For MongoDB we would be careful not to create these embeded documents, occasionally we might have a trade-off between redundancy and performance and then we might give priority to performance and maintain embedded documents.
* Minimize the redesign while extending the database.
* Avoid any bias toward any particular access pattern. # this would be avoided in MongoDB

Alternative schema design for blog application: (separate seek time for each different type of document - so pre-join)
        ![12_MOngoDB_id_2.JPG](img/12_MOngoDB_id_2.JPG)
        

Living without constraints:
* consistency is actually guranteed by constraints in RDBMS
* But in MongoDB, there is no such gurantee. This would be the responsibility of developer. However MongoDB embedded concept helps a bit to decrease the problem of not having constraints.


Living without transactions(ACID):
* But we do have atomic operation.
* Steps to take advantage of atomic operations:
    1. Restructure - so that you are working on a single document. If you do that then you are all set.
    2. Implement in software - like if we want to transfer some amount from one bank to another, then there couldn't be any transaction support across different RDBMS but the same problem/issue is resolved by software/application.To
    3. Tolerate a little bit of inconsistency - It is ok if one person sees a friend feed a little late (or not updated real-time)


One to One Relations:
* Examples:
    * Employee - Resume
    * Building - FloorPlan
    * Patient - Medical History
* Things to consider to decide the schema:
    * Frequency of access - (whether you access more of Employee data or Resume data, if employee are getting accessed more then is it ok to load all resume data if embeded, if embeded which one should be put inside to other depends upon access frequency)
    * Which of these are growing all the time or not growing:
        * One of the reason to keep these separate is that resume might be getting updated frequently and so you don't want to load employee details whenever there is an addition of employer in the resume.
        * And also if the size of resume is >16 MB then you might not be able to embed the employee in it.
    * Atomicity of the document - If you want to update both employee and resume details atomically then put that in single document.
    
    
Many to One Relations:
* Examples:
    * city - persons (new york city - 8 million people)
* Discussion for schema:
    * We ofcourse can't  embed person documents in city as there could be many persons and it would exceed that 16 MB condition.
    * Now tring to put city inside each person, but then there would be a lot of data redundancy which may lead to consistency issues.
* Solution - TRUE LINKING:
    - 2 collections (people[name, myCity] & city[city_id, <other details>]) - consistency would be guaranteed by application code.
    - But if we have one to few relations:
        * put the many embedded in the oneth relation.
        
When is it recommended to represent a one to many relationship in multiple collections?
    Whenever many is large
    
    
Many to Many Relations:
* Examples:
    * Books - Authors: [Few to Few]
    * Students - Teachers:
* Discussion for schema:-
    * Few to Few: 
        - 2 collections with references to both collections (as both are first level access objects)
        - embed books under author (avoid it unless performance constraint)
    * Many to few:
        - since teachers are generally few and they are actually required first so we create them alone
        - and in students document we add linking for teachers
        

Benifits of embedding:
    * Performance - proof-read performance (one trip to the DB)
    * General process of reading - it usually takes a lot of time to read the first byte and thereafter it reads every next byte pretty quickly.


Trees:
    * out of many db design problems, one is how you design underlying storage structure of tree type of data.
    * In MongoDB, we can define children/ancestors etc.
    
    
When to denormalize?
    * As we have seen, mongodb prefer embedded documents for performance point of view. It seems like we are denormalizing the data.
    * But till the point where we are duplicate the data, we don't open ourself to modification anamolies.
    
    
Handling blobs:
    - Storing large files:
        * by default 16 M.B. is the maximum storage size (document size limit)
        * But MongoDB also provide a special facility called GRIDFS that will break up large file into chunks and store those chunks into the collections and also store metadata about these chunks in secondary collection.
        * Say suppose we have a 100 MB video and we want to store the same in MongoDB
                gridfs.GridFS(db, "video.mp4")
        
        * 


.

### Sharding:

is a type of database partitioning that separates very large databases the into smaller, faster, more easily managed parts called data shards

data shards can be distributed across a number of much less expensive commodity servers. Data shards have comparatively little restriction as far as hardware and software requirements are concerned.

In some cases, database sharding can be done fairly simply. One common example is splitting a customer database geographically.

Data sharding can be a more complex process in some scenarios, however. Sharding a database that holds less structured data, for example, can be very complicated, and the resulting shards may be difficult to maintain.

Scale-up = increasing the power of CPU (scale vertical)

Scale-out = increasing the number of CPUs (scale horizontally) - MongoDB natively supports Scale-out concept (as no join is needed)

MongoDB also supports agile methodologies and performance needs of modern application.

MongoDB University course uses following things:
    1. MongoD (Database) - starting the server, use mongod and before that make sure to create directory \data\db
    2. Mongo (Shell) - when we run it, it makes connection to local database - now can run query on this prompt
    3. HTTP server (python, pymongo(mongo driver for python), bottle(web-framework) etc)
    
    
Datatypes supported in JSON - string,int,boolean,array,objects(nested jsons)

BSON - MongoDB stores data as BSON (Binary JSON) - This is the communication medium between mongoDB driver (at application side) and MOngoDB server. Below are the advantages of BSON-
    Light weight
    Traversable
    Efficient
    

Limitations of JSON: - BSON removed this limitations
    No support for date objects - must be stored as strings or numbers
    No difference between integers or float - all are stored as numbers
    

Indexing can be done on fields/keys with value of Array type as well.

Keys must be strings

MongoDB is schemaless, so if no value exist for a given key simply leave it.

Document size can't be more than 16 MB

Python Dict vs JSON:
    1. Ordering is maintained in JSON not in dict.
    2. 

### Bottle

bottle.template("hello_world", {"key1":"value1"}) # "hello_world.tpl" is the template file name and {{variable_name}} to populate the variable

## Performance

Performance of any computer application is derived by many things like:
1. Underlying hardware (CPU, memory, disk etc)
2. After hardware, algorithms comes into picture to determine the performance
3. And for database backed application, it is going to be the algorithms that satisfies queries.

There are two ways you can impact **latency and throughput** of database queries:
1. Add indexes to the collections.
2. Distribute the load across multiple servers using sharding.

Pluggable storage engines added in MOngoDB 3 (controls how data is going to store on disk)

Now database performance is actually on the plate of database administrators, but can developers deep dive into it and write their code keeping database performance in mind and they can debug the problems related to the performance and fix it. 

Pluggable storage engines: an interface between persistence storage (disks) and mongoDB
    * Flow - 
        ** python code interact with mongodb using pymongo where updation,deletion, insertion extra happens
        ** mongodb pass it to the pluggable storage engine
        ** which then passes it finally to the disk
        
        ** Now there could be case the storage engine would like to use memory to optimize the process. In other words the disk is quite slow
        
        ** So we introduce the pluggable concept and depending upon the type of storage engine plugged in the application, we get the performance characteristics.
        
   * Two storage engines shipped with mongodb by default:
       ** MMAP (default)
       ** WiredTiger - acquisition happened in 2014
       
   * what storage engine don't do?
       ** It doesn't have any control/impact on the communication between different mongodb servers in a cluster.
       ** Neither it has any impact on the APIs exposed to the python developer (or any other language developer)
    
  * The storage engine directly determines which of the following? Check all that apply.
      ** The data file format  (answer)
      ** Architecture of the cluster
      ** The wire protocol of drivers
      ** Format of indexes (answer)
      
      
* MMAP
    ** Also called version 1 as mongodb starts with this
    ** 